#Task3
##Anshika PH20C005

##Imports

In [1]:
from IPython.display import clear_output

!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip /content/nature_12K.zip

clear_output()

In [2]:
from PIL import Image

import os
from glob import glob
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary

# Dataloader

- Dataset Class for Setting up the data loading process
- Sections to fill in this script: `_init_transform()`

In [3]:
class inaturalist(Dataset):
    def __init__(self, root_dir, mode , transform = True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform      
        self._init_dataset()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.labels = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, 'train')))
        if self.mode == 'train': 
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'train', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        elif self.mode == 'val':
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'val', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        else:
            print("No Such Dataset Mode")
            return None
        
    def _init_transform(self):
        self.transform = transforms.Compose([transforms.Resize((227,227)), transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
        
    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB')
        label = self.labels[index]

        if self.transforms:
            img = self.transform(img)

        label = torch.tensor(label, dtype = torch.long)

        return img, label

    def __len__(self):
        return len(self.files)

# Model

- Class to define the model which we will use for training
- Stuff to fill in: The Architecture of your model, the `forward` function to define the forward pass

NOTE!: You are NOT allowed to use pretrained models for this task

In [4]:

class Classifier(nn.Module):
    def __init__(self, num_classes=10):
        super(Classifier, self).__init__()
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.l2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.l3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.l4 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc3= nn.Sequential(
            nn.Linear(4096, num_classes))
        
    def forward(self, x):
        out = self.l1(x)
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Training

- Sections to Fill: Define `loss` function, `optimizer` and model, `train` and `eval` functions and the training loop


## Hyperparameters

Feel free to change these hyperparams based on your machine's capactiy

In [5]:
batch_size = 32
epochs = 5
learning_rate = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

In [6]:
trainset = inaturalist(root_dir='/content/inaturalist_12K', mode='train',transform = True)
valset = inaturalist(root_dir='/content/inaturalist_12K', mode = 'val',transform = True)

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)

## Loss Function and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
model= Classifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


## Checkpoints

To save your model weights

In [8]:
checkpoint_dir = 'checkpoints'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

## Utility Functions

In [9]:
def get_model_summary(model, input_tensor_shape):
    summary(model, input_tensor_shape)

def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct/total

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Train

In [10]:
def train(model, trainloader, optimizer, criterion, device):
    '''
    Function to train the model for one epoch
    '''
    for image, label in trainloader:
      image = image.to(device)
      label = label.to(device)
      label=label-1
      optimizer.zero_grad()
      
      output = model(image)

      loss = criterion(output, label)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

## Eval

In [11]:
def eval(model, dataset, criterion, device):

    '''
    Function to validate the model after each epoch
    '''

    with torch.no_grad():
      correct=0
      total=0
      for image, label in dataset:
        image = image.to(device)
        label = label.to(device)
        label=label-1
        output= model(image)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
        del image,label,output
    print("Accuracy is ", (correct/total)* 100)


## Training

In [12]:
best_valid_loss = float('inf')

for epoch in range(epochs):
    
    start_time = time.monotonic()
    
    train(model,trainloader, optimizer,criterion, device)

    eval(model, valloader,criterion, device)


    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print("\n\n\n TIME TAKEN FOR THE EPOCH: {} mins and {} seconds".format(epoch_mins, epoch_secs))


print("OVERALL TRAINING COMPLETE")

Accuracy is  9.950000000000001



 TIME TAKEN FOR THE EPOCH: 23 mins and 45 seconds
Accuracy is  10.0



 TIME TAKEN FOR THE EPOCH: 23 mins and 35 seconds
Accuracy is  9.950000000000001



 TIME TAKEN FOR THE EPOCH: 27 mins and 49 seconds
Accuracy is  10.0



 TIME TAKEN FOR THE EPOCH: 27 mins and 22 seconds
Accuracy is  10.0



 TIME TAKEN FOR THE EPOCH: 27 mins and 34 seconds
OVERALL TRAINING COMPLETE
